# Heating plant

We have a dataset from a real heating plant located in a medium-sized city in Europe. The heating plant heats water and distributes the heat around the city. Our goal is to predict temperature of the returning water based on actual and historical power settings of the plant, output water temperature and the outside temperatures measured at different locations in the city.

## Approach to solution
- We will attempt to model the progression as separate time series predictions
- If we succeed in modeling the *temp_in*, we will attempt the same for the other variables.


## Data import
- we will consolidate the dataset in one data frame

In [ ]:
import pandas as pd
import plotly.express as px

frames = []
for i in range (0,10):
    frames.append(pd.read_excel("power_plant.xlsx", sheet_name=i))

In [ ]:
data = frames[0][['ts','power12']].merge(frames[1][['ts','power3']], on='ts').merge(frames[2][['ts','power4']], on='ts')
data = data.merge(frames[3][['ts','temp1']], on='ts')
data = data.merge(frames[4][['ts','temp2']], on='ts')
data = data.merge(frames[5][['ts','temp3']], on='ts')
data = data.merge(frames[6][['ts','temp4']], on='ts')
data = data.merge(frames[7][['ts','temp5']], on='ts')
data = data.merge(frames[8][['ts','temp_in']], on='ts')
data = data.merge(frames[9][['ts','temp_out']], on='ts')

data['ts'] = pd.to_datetime(data['ts'])
backup_frame = data.copy()
data.set_index('ts', inplace=True)
#data.index.freq = 'H'

px.scatter(data["temp_in"])


In [ ]:
data_interpolated = data.resample('H').interpolate(method='linear')  # Example for hourly interpolation
px.scatter(data_interpolated["temp_in"])

## Outlier filtering
The data is not clean, we have to solve a couple of relatively big issues:
1. Obviously wrong measurements in the *temp_in* data
2. The time stamps are not regular (missing measurements)

**Solutions**
1. Perform some kind of outlier filtering (Statistical - Standard deviation or Interquartile range / Logical - hard clip if we have a hypothesis)
2. 

In [ ]:
def filter_std(df, column):
    mean_temp = df[column].mean()
    std_temp = df[column].std()
    cutoff = std_temp * 3  # or 2, depending on how strict you want to be
    lower, upper = mean_temp - cutoff, mean_temp + cutoff
    return lower, upper

def filter_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    return lower, upper

# Calculate lower and upper bounds for outlier filtering

# HYPOTHESIS
# The wrong measurements are caused by malfunctions of sensors,
# therefore we will erase the most obvious mistakes by performing
# a hard clip based on the time series progression.

# You may play with the lower and upper bounds
# a bit to explore how the filtered data looks like.

#lower, upper = filter_std(data_interpolated, "temp_in")
lower = 40.
upper = 80.

# Perform filtering
data_filtered = data_interpolated.copy()

# Apply transformation
#data_filtered['temp_in'] = data_filtered['temp_in'].apply(lambda x: min(max(x, lower), upper))

# Remove data outside the bounds and replace with linear interpolation
mask = (data_filtered['temp_in'] >= lower) & (data_filtered['temp_in'] <= upper)
data_filtered = data_filtered[mask]
#data_filtered = data_filtered.interpolate(method='linear')
data_filtered = data_filtered.asfreq('H')  # Replace 'H' with your data's frequency
data_filtered['temp_in'].interpolate(method='linear', inplace=True)

missing_values = data_filtered['temp_in'].isnull().sum()
print(f"Number of missing values: {missing_values}")

# Visualize
fig = px.scatter(data_filtered["temp_in"])
fig.add_hline(y=lower, line_dash="dash", line_color="red")
fig.add_hline(y=upper, line_dash="dash", line_color="red")
fig.show()

## Filtering aftermath
Now that we have performed filtering and managed to insert the missing timestamps, now we can deal with the usual time series workflow.

In [ ]:
import statsmodels.api as sm
from plotly.subplots import make_subplots
import plotly.graph_objects as go

decomposition = sm.tsa.seasonal_decompose(data_filtered['temp_in'], model='multiplicative',period=2)  # or model='multiplicative'

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02)
# Trend
fig.add_trace(go.Scatter(x=trend.index, y=trend, mode='lines', name='Trend'), row=1, col=1)
# Seasonal
fig.add_trace(go.Scatter(x=seasonal.index, y=seasonal, mode='lines', name='Seasonal'), row=2, col=1)
# Residual
fig.add_trace(go.Scatter(x=residual.index, y=residual, mode='lines', name='Residual'), row=3, col=1)
# Settings
fig.update_layout(height=600, width=800, title_text="Time Series Decomposition")
fig.update_xaxes(title_text="Time", row=3, col=1)
fig.update_yaxes(title_text="Trend", row=1, col=1)
fig.update_yaxes(title_text="Seasonality", row=2, col=1)
fig.update_yaxes(title_text="Residual", row=3, col=1)
fig.show()

### Decomposition interpretation
- Right off the bat, we can see something strange in the seasonal component. This kind of plot suggests that there might be no seasonal process at all!
- There is also nothing in the residuals.
- Let's inspect the stationarity and autocorrelation to get some more information.

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(data_filtered['temp_in'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

# Interpret the p-value
if result[1] > 0.05:
    print("Series is not stationary")
else:
    print("Series is stationary")


The series is stationary according to ADF test then. That result could be expected based on the character of the *temp_in* progression.

# Plot ACF and PACF
We are looking for the first lag value where the ACF and PACF respectively are inside the confidence interval (blue area)
- PACF indicates the candidate for optimal *p* in the ARIMA model.
- ACF indicates the candidate *q*

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

plot_acf(data_filtered['temp_in'])
plot_pacf(data_filtered['temp_in'])
plt.show()

### ACF & PACF interpretation
- PACF indicates that the optimal $q = 3$
- The ACF plot however is problematic. The series autocorrelates with itself perfectly. That could mean a lot of things, but among them that there *might* be no underlying seasonal processes or patterns at all.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# ARIMA definition
p, d, q = 4, 0, 2
model = ARIMA(data_filtered['temp_in'], order=(p, d, q))
model_fit = model.fit()
print(model_fit.summary())

### ARIMA Summary
The model summary does not look good.
- the AIC and BIC are enormous. If you experiment with $p,d,q$, you may notice that they stay roughly the same level of disgusting.
- But our hopes still may be warranted, although the values are enormous, the model may yet work... Let's try to construct the series using forecasting:

In [ ]:
import numpy as np

y_all = data_filtered["temp_in"]
y_all = np.array(y_all)

train_size = int(len(y_all) * 0.8)  # 80% of data used for training
train, test = y_all[:train_size], y_all[train_size:]

# Generate ARIMA predictions using the fitted model and its parameters
predictions = model_fit.forecast(steps=len(test))

# Create a DataFrame for visualization
df = pd.DataFrame({
    'Actual': test,
    'Predicted': predictions,
    'Time': range(len(test))
})

# Calculate the difference
df['Difference'] = df['Actual'] - df['Predicted']

# Create a line plot for actual vs predicted
fig = px.line(df, x='Time', y=['Actual', 'Predicted'])

# Add a bar plot for the difference
fig.add_bar(x=df['Time'], y=df['Difference'], name='Difference')

# Update layout for clarity
fig.update_layout(
    title="ARIMA Model Predictions vs Actual",
    xaxis_title="Time",
    yaxis_title="Values",
    legend_title="Legend",
    barmode='overlay'  # Overlays the bar plot on the line plot
)

# Show the plot
fig.show()



### Lessons learned
- Our series was stationary according to the ADF test
- We had decent partial autocorrelation (PACF)
- the ACF was somewhat sketchy
- Decomposition only indicated that the series might not be seasonal, but that did not yet point to the series being unforecastable

Based on the result above, however, the model is an **obvious failure**.

#### Why is that?
The forecast has basically the shape of a series of mean values across the time frame. 
The seasonal elements us humans can kinda see behind the process (the oscillations during summer and winter) are hidden in the noise. To extract the underlying patterns, ranging from seasonality to direct inference from the other variables, we need to leverage the dependencies between them, and that is what we cannot do with (S)ARIMA(X) or simple LSTM's.

#### Key takeaways
This task was intended as a rather rough troubleshooting task, rather than a straightforward modeling assignment.
You were supposed to detect the missing timestamps, and decide how to deal with
1. The missing time stamps themselves, as you cannot fit them using either ARIMA or LSTM.
2. Once you inserted timestamps to adhere to the precribed frequency, how to deal with missing values of the series and outliers.
If you managed to get thus far and attempt to fit the ARIMA model, you are absolutely fantastic! It was a really hard assignment.
The straightforward time series prediction is just **not enough**.

As far as true solution goes, ARIMA is not the way. 
There are many ways to continue, if you had to persevere and find an approach which can serve at least as a baseline.

# Alternative approach

**Idea:** If we cut the time series into shorter segments, we may view the problem as static, thus time series modeling might be completely avoided!

We will thus perform a standard regression.
- This leverages the other variables in the prediction of the *temp_in*
- By performing more inferences over smaller time frames, we have a relatively high chance of accurately catching the time dependency implicitly...

In [ ]:
import random
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from datetime import timedelta

def rmse(predictions, true_values):
    return np.sqrt(mean_squared_error(predictions, true_values))

# We go back to the original dataset (data)
bf = backup_frame

#identify the gaps in the data that cause the trouble
bf['gap'] = (bf['ts'] - bf['ts'].shift(1)) != timedelta(hours=1)


I am going to segment the original data frame into more granular data frames, and we will use these for training our baseline model.

In [ ]:
# create a list of data frames based on the gap positions
attributes = ['power12', 'power3', 'power4', 'temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'temp_in', 'temp_out']

dflist = []

start = 0
for stop in range(1, len(bf)):
    if bf.iloc[stop]['gap']:
        dflist.append(bf[start:stop][attributes])
        start = stop
len(dflist)

w = 5 #window size
s = 1 #step
X_all = []
y_all = []

for df in dflist:
    for i in range(0, len(df)-w-1, s):
        X_all.append(df[i:i+w].values)
        y_all.append(df.iloc[i+w]['temp_in'])

In [ ]:
X_all = np.array(X_all)
y_all = np.array(y_all)

print (X_all.shape)
print (y_all.shape)

In [ ]:
import random
from sklearn.ensemble import GradientBoostingRegressor


mean_baseline = []
last_values_baseline = []
ml_model = []
iters = 20
train_size = int(0.9*len(X_all))

X_all_flat = X_all.reshape(X_all.shape[0], -1)
y_all_baseline = X_all_flat[:, -2]

for i in range(iters):
    all_data = list(zip(X_all_flat, y_all, y_all_baseline))
    random.shuffle(all_data)
    X_all_flat_rand, y_all_rand, y_all_rand_baseline = zip(*all_data)
    X_train = np.array(X_all_flat_rand[:train_size])
    y_train = np.array(y_all_rand[:train_size])
    X_test = np.array(X_all_flat_rand[train_size:])
    y_test = np.array(y_all_rand[train_size:])
    y_baseline = np.array(y_all_rand_baseline[train_size:])
    
    print ("Training iteration {}.".format(i+1))
    regr = GradientBoostingRegressor(n_estimators=500)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    
    m = np.mean(y_train)
    y_mean = np.array([m for i in range(len(y_test))])
    
    mean_baseline.append(rmse(y_mean, y_test))
    last_values_baseline.append(rmse(y_baseline, y_test))
    ml_model.append(rmse(y_pred, y_test))

In [ ]:
print ("Mean baseline: {} +- {}.".format(np.mean(mean_baseline), np.std(mean_baseline)))
print ("Last values baseline: {} +- {}.".format(np.mean(last_values_baseline), np.std(last_values_baseline)))
print ("ML model: {} +- {}.".format(np.mean(ml_model), np.std(ml_model)))

- Mean baseline: 6.280885842437059 +- 0.4750540060777046.
- Last values baseline: 1.4129028636032352 +- 0.40922030443042745.
- ML model: 1.3960461889178457 +- 0.34344185316048037.

Based on the results, the GBR approach is quite promising.
We could tweak the time frames and try to isolate seasonal components using more thorough segmentation.

Alternatively, we could try augmenting time series modeling with regular regression, or other advanced techniques.
The options are really endless.